In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Lib

In [1]:
!pip install rouge_score
!pip install -q contractions rouge
!pip install -q ipdb
!pip install gensim
!pip install tqdm

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=98fe7426682e1982f9099268a2d3b9f1609f44398cf88aa2c6030ecba8c0bf46
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 60.4 MB/s eta 0:00:00


In [3]:
import warnings
import re
import time
import json
import random
from typing import List, Dict, Tuple
from functools import partial

import numpy as np
import pandas as pd
from tqdm import tqdm
from tqdm.auto import tqdm
from scipy.sparse.linalg import svds

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
import contractions

from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import gensim
import rouge
from rouge_score import rouge_scorer

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score
)

import torch
from transformers import BertTokenizer, BertModel
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout

import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from textwrap import wrap
from IPython.core.display import display, HTML
import difflib


warnings.filterwarnings('ignore')

%matplotlib inline


nltk_resources = ["punkt", "stopwords", "punkt_tab", "wordnet", "omw-1.4"]
print("Checking and downloading NLTK resources...")
for resource in nltk_resources:
    try:
        nltk.data.find(resource)
    except LookupError:
        print(f"Downloading {resource}...")
        nltk.download(resource)

networkx = nx

import nltk
from nltk.tokenize import sent_tokenize
from rouge_score import rouge_scorer
from multiprocessing import Pool, cpu_count
from functools import partial
import warnings
warnings.filterwarnings('ignore')
import re
import nltk
import contractions
from typing import List, Optional
from tqdm.auto import tqdm

Checking and downloading NLTK resources...


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


# Summarization


## Importing data

In [ ]:
file_path = "drive/MyDrive/train-stats.jsonl"
N = 50000  # number of rows to extract randomly
random.seed(42)
reservoir = []
with open(file_path, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        data = json.loads(line)

        if i < N:
            reservoir.append(data)
        else:
            j = random.randint(0, i)
            if j < N:
                reservoir[j] = data

df_train = pd.DataFrame(reservoir)
df_train.head()

,url,archive,title,date,text,summary,compression,coverage,density,compression_bin,coverage_bin,density_bin
0,http://www.theguardian.com/us-news/2015/jul/24...,https://web.archive.org/web/2015072419id_/http...,Obama's Kenya agenda both personal and politic...,2015072419,A little girl in a white dress waited at the b...,President arrives in Nairobi amid high securit...,54.620690,0.724138,1.344828,high,low,abstractive
1,http://www.nytimes.com/2014/06/04/world/europe...,https://web.archive.org/web/2014060319id_/http...,New Fighting Reported in Eastern Ukraine,2014060319,"LUHANSK, Ukraine — Ukraine’s minister of inter...",Ukraine’s interior minister said that separati...,44.750000,0.850000,1.550000,high,medium,mixed
2,http://www.nytimes.com/2006/04/23/business/you...,http://web.archive.org/web/20060909062911id_/h...,Investors vs. Pfizer: Guess Who Has the Guns?,20060909062911,IF outsized executive pay has indeed become a ...,The battle between Pfizer Inc.'s investors and...,33.880000,1.000000,11.720000,medium,high,extractive
3,http://blogs.aljazeera.com/blog/africa/zanziba...,https://web.archive.org/web/2012100119id_/http...,Zanzibar plagued by heroin addiction,2012100119,There is very little data available on the ext...,With easy access and heroin costing just $2 fo...,20.407407,0.925926,5.296296,medium,medium,mixed
4,http://www.washingtonpost.com/blogs/ask-the-po...,https://web.archive.org/web/2013042119id_/http...,Opting out of school tests: Readers respond,2013042119,Lyndsey Layton’s terrific story this morning o...,A cross-section of reaction to the growing res...,50.071429,0.785714,2.357143,high,medium,mixed


In [ ]:
file_path = "drive/MyDrive/dev-stats.jsonl"
N = 1000  # number of rows to extract randomly
random.seed(42)
reservoir = []
with open(file_path, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        data = json.loads(line)

        if i < N:
            reservoir.append(data)
        else:
            j = random.randint(0, i)
            if j < N:
                reservoir[j] = data

df_val = pd.DataFrame(reservoir)
df_val.head()

,url,archive,title,date,text,summary,compression,coverage,density,compression_bin,coverage_bin,density_bin
0,http://www.telegraph.co.uk/dance/what-to-see/p...,http://web.archive.org/web/20160805103723id_/h...,Picture exclusive: Inside the Royal Ballet's c...,20160805103723,"‘This isn’t a selfish thing,’ he tells me, awa...","Deep below stairs at the Royal Opera House, a ...",7.055556,0.444444,0.555556,low,low,abstractive
1,http://abcnews.go.com/blogs/entertainment/2013...,https://web.archive.org/web/2013072519id_/http...,Shakira’s Amazing Bikini Body Just 6 Months Af...,2013072519,Looking at recent pictures of Shakira vacation...,Looking at recent pictures of Shakira vacation...,1.522059,0.933824,13.610294,low,medium,extractive
2,http://www.cnn.com/2014/09/23/opinion/suzman-f...,https://web.archive.org/web/2014092319id_/http...,Climate change: How to help farmers adapt (Opi...,2014092319,"Severe weather: Flood, fire and drought\n\nSev...",Mark Suzman says it's not enough just to try t...,36.192308,0.769231,1.076923,high,low,abstractive
3,http://www.nytimes.com/2013/03/17/fashion/ben-...,https://web.archive.org/web/2013031819id_/http...,"Ben Affleck, the Leader of the Pack",2013031819,"But that is where Ben Affleck could be found, ...",Showbizâs leading man of the moment is Ben A...,44.714286,0.857143,3.942857,high,medium,mixed
4,http://www.nytimes.com/2015/09/05/sports/baseb...,https://web.archive.org/web/2015090419id_/http...,Scott Boras and the Mets Disagree About Matt H...,2015090419,"MIAMI — Over the winter, as the Mets prepared ...",Five months into Harvey’s first season back fr...,31.903226,0.935484,5.129032,medium,medium,mixed


## Extractive Sentence Selection

For each row, the sentences in the original text are compared with the sentences in the summary using the ROUGE-L metric. The sentences in the text which are sufficiently similar to the summary sentences are identified and they are marks as “extractive” by creating a list of boolean values.


In [ ]:
def align_text_to_summary(row, threshold=0.4):
    '''
    Align text to summary using ROUGE-L and return a list of booleans,
    indicating which sentences are selected
    '''

    text = row['text']
    summary = row['summary']

    # Sentence segmentation
    text_sentences = sent_tokenize(str(text))
    summary_sentences = sent_tokenize(str(summary))

    if not text_sentences or not summary_sentences:
       return [False] * len(text_sentences)

    # Initialize ROUGE scorer
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

    # Array to track selected sentences
    selected = [False] * len(text_sentences)

    # For each sentence in the summary, find the best matching sentence in the text
    for sum_sent in summary_sentences:
        best_score = 0.0
        best_idx = -1

        for idx, txt_sent in enumerate(text_sentences):
            # Compute ROUGE-L F-Measure
            scores = scorer.score(sum_sent, txt_sent)
            rouge_l_f = scores['rougeL'].fmeasure

            if rouge_l_f > best_score:
                best_score = rouge_l_f
                best_idx = idx

        # If the score exceeds the threshold, mark the sentence as selected
        if best_idx >= 0 and best_score >= threshold:
            selected[best_idx] = True

    return selected

The extraction process is applied to every row in the DataFrame, storing these lists in a new column called extractive_labels. This allows to see, for each text, which sentences are considered representative of the summary.

Finally, the training set is filtered to keep only the documents that contain at least two extractive sentences.

In [ ]:
def process_dataframe(df, threshold=0.4):
    '''
    Process a DataFrame with extractive alignment and return it with an added
    'extractive_labels' column.
    '''

    df_result = df.copy()
    extractive_labels = []

    for _, row in df_result.iterrows():
        labels = align_text_to_summary(row, threshold=threshold)
        extractive_labels.append(labels)

    df_result['extractive_labels'] = extractive_labels
    return df_result


# Filtering of training set
df_train_extractive = process_dataframe(df_train, threshold=0.4)

df_train = df_train_extractive[
    df_train_extractive['extractive_labels'].apply(lambda x: sum(x) >= 2)
]

print(f"Number of training documents with at least 2 extractive sentences: {len(df_train)}")

Number of training documents with at least 2 extractive sentences: 6367


## Enhanced TextRank with BERT

In [ ]:
# Create Newsroom-specific stopwords
def create_stopwords_newsroom(
    lst_langs: List[str] = ["english"],
    lst_add_words: Optional[List[str]] = None,
    lst_keep_words: Optional[List[str]] = None
) -> List[str]:
    """
    Create a custom stopword list for Newsroom articles.
    """
    if lst_add_words is None:
        lst_add_words = [
            "said", "say", "says", "mr", "ms", "mrs", "also", "new", "news",
            "report", "reports", "reporter", "cnn", "wa", "ha", "would", "could",
            "us", "one", "two", "first", "last", "like", "just", "get", "got"
        ]
    if lst_keep_words is None:
        lst_keep_words = []

    lst_stopwords = set()
    for lang in lst_langs:
        lst_stopwords.update(nltk.corpus.stopwords.words(lang))
    lst_stopwords.update(lst_add_words)
    lst_stopwords.difference_update(lst_keep_words)

    return sorted(lst_stopwords)


# Preprocess Newsroom text
def preprocess_text_newsroom(
    txt: str,
    lst_stopwords: Optional[List[str]] = None,
    lst_regex: Optional[List[str]] = None,
    remove_punct: bool = True,
    lower: bool = True,
    fix_contractions: bool = True,
    stemm: bool = False,
    lemm: bool = True
) -> str:
    """
    Preprocess text for Newsroom articles:
    regex cleaning, punctuation removal, lowercasing,
    contraction expansion, stopword removal, stemming/lemmatization.
    """
    # Apply regex cleaning
    if lst_regex:
        for regex in lst_regex:
            txt = re.sub(regex, '', txt)

    # Separate sentences properly
    txt = re.sub(r'\.(?=[^ \W\d])', '. ', str(txt))

    # Remove punctuation/non-alphanumeric characters
    if remove_punct:
        txt = re.sub(r'[^\w\s]', '', txt)

    # Strip extra spaces
    txt = " ".join([w.strip() for w in txt.split()])

    # Lowercase
    if lower:
        txt = txt.lower()

    # Expand contractions
    if fix_contractions:
        txt = contractions.fix(txt)

    # Tokenize
    tokens = txt.split()

    # Stemming
    if stemm:
        stemmer = nltk.stem.porter.PorterStemmer()
        tokens = [stemmer.stem(w) for w in tokens]

    # Lemmatization
    if lemm:
        lemmatizer = nltk.stem.WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(w) for w in tokens]

    # Remove stopwords
    if lst_stopwords:
        tokens = [w for w in tokens if w not in lst_stopwords]

    # Reconstruct text
    return " ".join(tokens)


# Apply preprocessing to a DataFrame column
def add_preprocessed_text(
    data: pd.DataFrame,
    column: str,
    lst_stopwords: Optional[List[str]] = None,
    lst_regex: Optional[List[str]] = None,
    remove_punct: bool = True,
    lower: bool = True,
    fix_contractions: bool = True,
    stemm: bool = False,
    lemm: bool = True,
    remove_na: bool = True
) -> pd.DataFrame:
    """
    Apply Newsroom preprocessing to a DataFrame column.
    Adds a cleaned column with the suffix '_clean'.
    """
    dtf = data.copy()

    # Keep only non-null rows
    dtf = dtf[pd.notnull(dtf[column])]

    # Preprocess text
    tqdm.pandas(desc='Cleaning text')
    dtf[column + "_clean"] = dtf[column].progress_apply(
        lambda x: preprocess_text_newsroom(
            x,
            lst_stopwords=lst_stopwords,
            lst_regex=lst_regex,
            remove_punct=remove_punct,
            lower=lower,
            fix_contractions=fix_contractions,
            stemm=stemm,
            lemm=lemm
        )
    )

    # Remove empty strings
    dtf["check"] = dtf[column + "_clean"].progress_apply(lambda x: len(x))
    if dtf["check"].min() == 0:
        print("--- found empty texts ---")
        if remove_na:
            dtf = dtf[dtf["check"] > 0]

    return dtf.drop("check", axis=1)

In [ ]:
# Create stopwords
lst_stopwords = create_stopwords_newsroom()

# Preprocess training and validation corpus
df_train_clean = add_preprocessed_text(df_train, "text", lst_stopwords=lst_stopwords)
df_val_clean = add_preprocessed_text(df_val, "text", lst_stopwords=lst_stopwords)

Cleaning text:   0%|          | 0/6367 [00:00<?, ?it/s]

Cleaning text:   0%|          | 0/6367 [00:00<?, ?it/s]

Cleaning text:   0%|          | 0/1000 [00:00<?, ?it/s]

Cleaning text:   0%|          | 0/1000 [00:00<?, ?it/s]

We apply Enhanced Extractive Summarization TextRank which combines contextualized BERT embeddings, TF-IDF weighting, and the TextRank algorithm.



### TF-IDF weighting

We build a TF-IDF calculator that ranks how important each word is in a document relative to the whole corpus.
It computes IDF to capture word rarity and TF to measure frequency within the document.
Multiplying them gives a score that emphasizes terms that appear often in the text but are uncommon elsewhere.

In [ ]:
# Fit IDF on corpus
def fit_idf(corpus: List[str]) -> Dict[str, float]:
    vectorizer = TfidfVectorizer(
        lowercase=True,
        token_pattern=r'\b\w+\b'
    )
    vectorizer.fit(corpus)
    return dict(zip(vectorizer.get_feature_names_out(), vectorizer.idf_))


# Term Frequency
def tf(term: str, document: List[str]) -> float:
    term = term.lower()
    doc_lower = [w.lower() for w in document]
    total_terms = len(doc_lower)
    if total_terms == 0:
        return 0.0
    return doc_lower.count(term) / total_terms


# Inverse Document Frequency
def idf(term: str, idf_dict: Dict[str, float]) -> float:
    return idf_dict.get(term.lower(), 0.0)


# TF-IDF score
def tfidf(term: str, document: List[str], idf_dict: Dict[str, float]) -> float:
    return tf(term, document) * idf(term, idf_dict)

### BERT Embedder

The embedder produces contextual BERT vectors for each word, then weights them using TF-IDF to reflect term importance.
By averaging these weighted embeddings, it outputs a single sentence vector that captures both semantic meaning and document-specific relevance.

In [ ]:
# Setup BERT model and tokenizer
def init_bert(model_name: str = 'bert-base-uncased'):
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertModel.from_pretrained(model_name)
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    return tokenizer, model, device

# Generate contextual embeddings for each word in a sentence
def get_contextual_embeddings(sentence: str, tokenizer, model, device) -> Dict[str, np.ndarray]:
    tokens = tokenizer.tokenize(sentence.lower())
    if len(tokens) > 512:
        tokens = tokens[:512]
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokens)
    tokens_tensor = torch.tensor([indexed_tokens]).to(device)

    with torch.no_grad():
        outputs = model(tokens_tensor)
        hidden_states = outputs.last_hidden_state

    embeddings = hidden_states[0].cpu().numpy()
    word_embeddings = {}
    for token, embedding in zip(tokens, embeddings):
        word = token.replace('##', '')
        if word in word_embeddings:
            word_embeddings[word] = (word_embeddings[word] + embedding) / 2
        else:
            word_embeddings[word] = embedding
    return word_embeddings

# Compute weighted sentence vector using TF-IDF
def get_weighted_sentence_vector(sentence: str, document_words: List[str], idf_dict: Dict[str, float],
                                 tokenizer, model, device) -> np.ndarray:
    """
    sentence: text of the sentence
    document_words: list of all words in the document for TF calculation
    idf_dict: precomputed {term: idf} dictionary
    """
    word_embeddings = get_contextual_embeddings(sentence, tokenizer, model, device)
    weighted_vectors = []

    for word, embedding in word_embeddings.items():
        # TF calculation
        term_count = document_words.count(word.lower())
        total_terms = len(document_words)
        tf = term_count / total_terms if total_terms > 0 else 0.0

        # TF-IDF weight
        idf_weight = idf_dict.get(word.lower(), 0.0)
        tfidf_weight = tf * idf_weight

        weighted_vectors.append(embedding * tfidf_weight)

    return np.mean(weighted_vectors, axis=0) if weighted_vectors else np.zeros(768)

### TextRank Algorithm

We build a similarity graph from sentence vectors, rank sentences with PageRank, and extract the top ones.
The selected sentences are then reordered according to their original position to produce a coherent extractive summary.

In [ ]:
# Build similarity matrix
def build_similarity_matrix(sentence_vectors: List[np.ndarray]) -> np.ndarray:
    # Given a list of sentence vectors, returns a cosine similarity matrix.

    return cosine_similarity(sentence_vectors)

# Apply TextRank algorithm
def textrank(similarity_matrix: np.ndarray, damping_factor: float = 0.85) -> np.ndarray:
    # Given a similarity matrix, computes TextRank scores for each sentence.

    # Build weighted graph
    nx_graph = nx.from_numpy_array(similarity_matrix)

    # Apply PageRank
    scores = nx.pagerank(nx_graph, alpha=damping_factor, max_iter=100, tol=1e-6)

    # Return scores as array in original sentence order
    return np.array([scores[i] for i in range(len(scores))])

# Extract top sentences as summary
def extract_summary(sentences: List[str], scores: np.ndarray, num_sentences: int) -> Tuple[List[str], List[int]]:
    # Selects top-N sentences based on scores and returns them in original order.

    ranked_indices = np.argsort(scores)[::-1]
    top_indices = ranked_indices[:num_sentences]
    top_indices_sorted = sorted(top_indices)
    summary_sentences = [sentences[i] for i in top_indices_sorted]
    return summary_sentences, top_indices_sorted

### Extractive Summarize

We set up the TF-IDF calculator, BERT embedder, and TextRank module, then turn sentences into weighted embeddings and rank them using PageRank.
The top sentences are extracted in their original order, and ROUGE is used to evaluate the summary against a reference.

In [ ]:
# Document preprocessing
def preprocess_document(text: str) -> Tuple[List[str], List[str]]:
    # Splits text into sentences and tokenizes all words (lowercased)
    sentences = sent_tokenize(text)
    all_words = word_tokenize(text.lower())
    return sentences, all_words

# ROUGE evaluation
def evaluate_rouge(generated: str, reference: str) -> Dict[str, float]:
    # Compute ROUGE-1, ROUGE-2, ROUGE-L F1 scores

    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, generated)
    rouge_results = {
        'ROUGE-1': scores['rouge1'].fmeasure,
        'ROUGE-2': scores['rouge2'].fmeasure,
        'ROUGE-L': scores['rougeL'].fmeasure
    }
    return rouge_results

# Summarize document
def summarize_document(
    document: str,
    idf_dict: Dict[str, float],
    tokenizer,
    bert_model,
    device,
    target_summary_length: int = None,
    ground_truth: str = None,
    damping_factor: float = 0.85
) -> Dict:
    # Complete extractive summarization pipeline

    sentences, document_words = preprocess_document(document)

    # Determine number of sentences to extract
    if target_summary_length is None:
        if ground_truth is not None:
            gt_sentences = sent_tokenize(ground_truth)
            target_summary_length = len(gt_sentences)
        else:
            target_summary_length = max(1, len(sentences) // 4)  # default 25%

    # Compute weighted sentence vectors (BERT + TF-IDF)
    sentence_vectors = []
    for sent in sentences:
        vec = get_weighted_sentence_vector(
            sent, document_words, idf_dict, tokenizer, bert_model, device
        )
        sentence_vectors.append(vec)

    # Compute similarity matrix
    similarity_matrix = build_similarity_matrix(sentence_vectors)

    # Apply TextRank
    scores = textrank(similarity_matrix, damping_factor=damping_factor)

    # Extract top sentences
    summary_sentences, selected_indices = extract_summary(
        sentences, scores, target_summary_length
    )

    summary = ' '.join(summary_sentences)
    results = {'summary': summary}

    # ROUGE evaluation
    if ground_truth is not None:
        rouge_scores = evaluate_rouge(summary, ground_truth)
        results['rouge_scores'] = rouge_scores

    return results

### Results

In [ ]:
# Prepare corpus and IDF
newsroom_corpus = df_train_clean["text_clean"].tolist()
idf_dict = fit_idf(newsroom_corpus)

# Initialize BERT once
tokenizer, bert_model, device = init_bert('bert-base-uncased')

# Lists for ROUGE metrics
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []


# Loop over all documents
for idx, row in df_val.iterrows():
    doc = row["text"]
    reference = row["summary"]

    # Generate summary
    results = summarize_document(
        document=doc,
        idf_dict=idf_dict,
        tokenizer=tokenizer,
        bert_model=bert_model,
        device=device,
        ground_truth=reference
    )

    # Save ROUGE scores
    rouge_scores = results.get('rouge_scores', {})
    rouge1_scores.append(rouge_scores.get('ROUGE-1', 0.0))
    rouge2_scores.append(rouge_scores.get('ROUGE-2', 0.0))
    rougeL_scores.append(rouge_scores.get('ROUGE-L', 0.0))

# Calculate final average ROUGE on test set
avg_rouge1 = np.mean(rouge1_scores)
avg_rouge2 = np.mean(rouge2_scores)
avg_rougeL = np.mean(rougeL_scores)

print(f"\nFinal ROUGE scores on our test set:")
print(f"  Average ROUGE-1: {avg_rouge1:.4f}")
print(f"  Average ROUGE-2: {avg_rouge2:.4f}")
print(f"  Average ROUGE-L: {avg_rougeL:.4f}")


Final ROUGE scores on our test set:
  Average ROUGE-1: 0.2269
  Average ROUGE-2: 0.0892
  Average ROUGE-L: 0.1745


In [ ]:
def utils_split_sentences(a, b):
    # find clean matches
    match = difflib.SequenceMatcher(isjunk=None, a=a, b=b, autojunk=True)
    lst_match = [block for block in match.get_matching_blocks() if block.size > 20]

    # difflib didn't find any match
    if len(lst_match) == 0:
        lst_a, lst_b = nltk.sent_tokenize(a), nltk.sent_tokenize(b)

    # work with matches
    else:
        first_m, last_m = lst_match[0], lst_match[-1]

        # Parameter "a"
        string = a[0 : first_m.a]
        lst_a = [t for t in nltk.sent_tokenize(string)]
        for n in range(len(lst_match)):
            m = lst_match[n]
            string = a[m.a : m.a+m.size]
            lst_a.append(string)
            if n+1 < len(lst_match):
                next_m = lst_match[n+1]
                string = a[m.a+m.size : next_m.a]
                lst_a = lst_a + [t for t in nltk.sent_tokenize(string)]
            else:
                break
        string = a[last_m.a+last_m.size :]
        lst_a = lst_a + [t for t in nltk.sent_tokenize(string)]

        # Parameter "b"
        string = b[0 : first_m.b]
        lst_b = [t for t in nltk.sent_tokenize(string)]
        for n in range(len(lst_match)):
            m = lst_match[n]
            string = b[m.b : m.b+m.size]
            lst_b.append(string)
            if n+1 < len(lst_match):
                next_m = lst_match[n+1]
                string = b[m.b+m.size : next_m.b]
                lst_b = lst_b + [t for t in nltk.sent_tokenize(string)]
            else:
                break
        string = b[last_m.b+last_m.size :]
        lst_b = lst_b + [t for t in nltk.sent_tokenize(string)]

    return lst_a, lst_b


In [ ]:
def display_string_matching(a, b, both=True, sentences=True, titles=[]):
    if sentences is True:
        lst_a, lst_b = utils_split_sentences(a, b)
    else:
        lst_a, lst_b = a.split(), b.split()

    # highlight parameter "a"
    first_text = []
    for i in lst_a:
        if re.sub(r'[^\w\s]', '', i.lower()) in [re.sub(r'[^\w\s]', '', z.lower()) for z in lst_b]:
            first_text.append('<span style="background-color:rgba(255,215,0,0.3);">' + i + '</span>')
        else:
            first_text.append(i)
    first_text = ' '.join(first_text)

    # highlight parameter "b"
    second_text = []
    if both is True:
        for i in lst_b:
            if re.sub(r'[^\w\s]', '', i.lower()) in [re.sub(r'[^\w\s]', '', z.lower()) for z in lst_a]:
                second_text.append('<span style="background-color:rgba(255,215,0,0.3);">' + i + '</span>')
            else:
                second_text.append(i)
    else:
        second_text.append(b)
    second_text = ' '.join(second_text)

    # concatenate
    if len(titles) > 0:
        first_text = "<strong>"+titles[0]+"</strong><br>"+first_text
    if len(titles) > 1:
        second_text = "<strong>"+titles[1]+"</strong><br>"+second_text
    else:
        second_text = "---"*65+"<br><br>"+second_text
    final_text = first_text +'<br><br>'+ second_text
    return final_text

In [ ]:
i = 14  # index of the document to visualize

# Get the real summary and the generated summary
reference = df_val.iloc[i]["summary"]
doc = df_val.iloc[i]["text"]

# Generation of the predicted summary for this document
results = summarize_document(
    document=doc,
    idf_dict=idf_dict,
    tokenizer=tokenizer,
    bert_model=bert_model,
    device=device,
    ground_truth=reference
)

# Highlight matching words between real and predicted summary
highlight_match = display_string_matching(
    a=reference,
    b=results['summary'],
    both=True,
    sentences=False,
    titles=["Real Summary", "Predicted Summary"]
)

display(HTML(highlight_match))


## TextRank without BERT

Extractive summarization approach that ranks sentences using TextRank without leveraging BERT embeddings.

### Preprocessing

In [ ]:
# Create stopwords
lst_stopwords = create_stopwords_newsroom()

# Preprocess training and validation corpus
df_train_clean = add_preprocessed_text(df_train, "text", lst_stopwords=lst_stopwords)
df_val_clean = add_preprocessed_text(df_val, "text", lst_stopwords=lst_stopwords)

Cleaning text:   0%|          | 0/6367 [00:00<?, ?it/s]

Cleaning text:   0%|          | 0/6367 [00:00<?, ?it/s]

Cleaning text:   0%|          | 0/1000 [00:00<?, ?it/s]

Cleaning text:   0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
def get_weighted_sentence_vector(
    sentence: str,
    document_words: List[str],
    idf_dict: Dict[str, float]
) -> np.ndarray:
    """
    Compute sentence vector using TF-IDF (no embeddings, no BERT)
    """

    sentence_words = sentence.lower().split()
    vector = []

    total_terms = len(document_words)

    for word in sentence_words:
        # TF
        term_count = document_words.count(word)
        tf = term_count / total_terms if total_terms > 0 else 0.0

        # IDF
        idf = idf_dict.get(word, 0.0)

        vector.append(tf * idf)

    return np.array(vector)

In [ ]:
def compute_sentence_tfidf_vectors(sentences):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(sentences)
    return tfidf_matrix

Redefintion of the summarization function without BERT

In [ ]:
def summarize_document(
    document: str,
    idf_dict: Dict[str, float],
    target_summary_length: int = None,
    ground_truth: str = None,
    damping_factor: float = 0.85
) -> Dict:
    """
    Extractive summarization using TF-IDF + TextRank (no BERT)
    """

    # Preprocessing
    sentences, document_words = preprocess_document(document)

    # Determine number of sentences to extract
    if target_summary_length is None:
        if ground_truth is not None:
            target_summary_length = len(sent_tokenize(ground_truth))
        else:
            target_summary_length = max(1, len(sentences) // 4)

    # Compute sentence vectors (TF-IDF only)
    sentence_vectors = compute_sentence_tfidf_vectors(sentences)

    # Similarity matrix
    similarity_matrix = build_similarity_matrix(sentence_vectors)

    # TextRank
    scores = textrank(similarity_matrix, damping_factor=damping_factor)

    # Select top sentences
    summary_sentences, selected_indices = extract_summary(
        sentences,
        scores,
        target_summary_length
    )

    summary = " ".join(summary_sentences)
    results = {"summary": summary}

    # ROUGE evaluation
    if ground_truth is not None:
        results["rouge_scores"] = evaluate_rouge(summary, ground_truth)

    return results

### Results

In [ ]:
# Build IDF dictionary on training corpus
newsroom_corpus = df_train_clean["text_clean"].tolist()
idf_dict = fit_idf(newsroom_corpus)

# Lists for ROUGE metrics
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []

# Loop over all documents
for idx, row in df_val.iterrows():
    doc = row["text"]
    reference = row["summary"]

    # Generate summary (TF-IDF + TextRank)
    results = summarize_document(
        document=doc,
        idf_dict=idf_dict,
        ground_truth=reference
    )

    # Save ROUGE scores
    rouge_scores = results.get("rouge_scores", {})
    rouge1_scores.append(rouge_scores.get("ROUGE-1", 0.0))
    rouge2_scores.append(rouge_scores.get("ROUGE-2", 0.0))
    rougeL_scores.append(rouge_scores.get("ROUGE-L", 0.0))

# Calculate final average ROUGE on the set
avg_rouge1 = np.mean(rouge1_scores)
avg_rouge2 = np.mean(rouge2_scores)
avg_rougeL = np.mean(rougeL_scores)

print("\nFinal ROUGE scores on our test set:")
print(f"  Average ROUGE-1: {avg_rouge1:.4f}")
print(f"  Average ROUGE-2: {avg_rouge2:.4f}")
print(f"  Average ROUGE-L: {avg_rougeL:.4f}")


Final ROUGE scores on our test set:
  Average ROUGE-1: 0.2288
  Average ROUGE-2: 0.0856
  Average ROUGE-L: 0.1760


We visually compare a reference summary with a predicted one by highlighting matching words, showing side-by-side which parts overlap and helping assess the extractive quality of the LSA model.

In [ ]:
i = 14  # index of the document to visualize

# Get the real summary and the generated summary
reference = df_val.iloc[i]["summary"]
doc = df_val.iloc[i]["text"]

# Generation of the predicted summary for this document
results = summarize_document(
        document=doc,
        idf_dict=idf_dict,
        ground_truth=reference
    )

# Highlight matching words between real and predicted summary
highlight_match = display_string_matching(
    a=reference,
    b=results['summary'],
    both=True,
    sentences=False,
    titles=["Real Summary", "Predicted Summary"]
)

display(HTML(highlight_match))

# Classification


## Importing data

In [ ]:
train_df=pd.read_csv("drive/MyDrive/train.csv")
train_df.head()

,Class Index,Title,Description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."


In [ ]:
test_df=pd.read_csv("drive/MyDrive/test.csv")
test_df.head()

,Class Index,Title,Description
0,3,Fears for T N pension after talks,Unions representing workers at Turner Newall...
1,4,The Race is On: Second Private Team Sets Launc...,"SPACE.com - TORONTO, Canada -- A second\team o..."
2,4,Ky. Company Wins Grant to Study Peptides (AP),AP - A company founded by a chemistry research...
3,4,Prediction Unit Helps Forecast Wildfires (AP),AP - It's barely dawn when Mike Fitzpatrick st...
4,4,Calif. Aims to Limit Farm-Related Smog (AP),AP - Southern California's smog-fighting agenc...


## Logistic Regression & Linear Support Vector Classifier


### Preprocessing

In [ ]:
# Combine Title and Description into a single feature
train_df["text"] = train_df["Title"] + " " + train_df["Description"]
test_df["text"] = test_df["Title"] + " " + test_df["Description"]

# Extract X (text) and y (label)
X_train = train_df["text"]
y_train = train_df["Class Index"]

X_test = test_df["text"]
y_test = test_df["Class Index"]

Data Cleaning of the text by lowercasing, removing non-alphabetic characters, and collapsing extra whitespace to obtain clean input text for vectorization.

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z\s]", "", text)  # Removes everything except letters and spaces
    text = re.sub(r"\s+", " ", text).strip()
    return text

X_train_clean = X_train.apply(clean_text)
X_test_clean = X_test.apply(clean_text)

We create a TF-IDF representation of the text using unigrams and bigrams, filtering very rare and very frequent terms, limiting the vocabulary size, and applying sublinear term frequency scaling to improve robustness.



In [ ]:
vectorizer = TfidfVectorizer(
    lowercase=True,
    ngram_range=(1,2),
    min_df=2,
    max_df=0.9,
    max_features=200,
    sublinear_tf=True
)

X_train_vec = vectorizer.fit_transform(X_train_clean)
X_test_vec = vectorizer.transform(X_test_clean)

### Training Phase & Results

Train a logistic regression classifier and evaluate its performance.

In [ ]:
clf_lr = LogisticRegression(
    max_iter=2000,
    solver="lbfgs", # optimizer suitable for multinomial logistic regression
    C=1.0, # Regularization strength
    n_jobs=-1
)

clf_lr.fit(X_train_vec, y_train)

preds_lr = clf_lr.predict(X_test_vec)

print("Accuracy Logistic Regression:", accuracy_score(y_test, preds_lr))
print(classification_report(y_test, preds_lr))

Accuracy Logistic Regression: 0.7267105263157895
              precision    recall  f1-score   support

           1       0.74      0.72      0.73      1900
           2       0.77      0.83      0.80      1900
           3       0.73      0.67      0.70      1900
           4       0.66      0.69      0.68      1900

    accuracy                           0.73      7600
   macro avg       0.73      0.73      0.73      7600
weighted avg       0.73      0.73      0.73      7600



Fit a linear SVM and report its predictive performance.

In [ ]:
clf_svc = LinearSVC(
    C=1.0,
    max_iter=5000
)

clf_svc.fit(X_train_vec, y_train)

preds_svc = clf_svc.predict(X_test_vec)

print("Accuracy SVC:", accuracy_score(y_test, preds_svc))
print(classification_report(y_test, preds_svc))

Accuracy SVC: 0.7247368421052631
              precision    recall  f1-score   support

           1       0.75      0.70      0.72      1900
           2       0.75      0.84      0.79      1900
           3       0.73      0.66      0.70      1900
           4       0.67      0.69      0.68      1900

    accuracy                           0.72      7600
   macro avg       0.72      0.72      0.72      7600
weighted avg       0.72      0.72      0.72      7600



## 1D Convolutional Neural Network

### Preprocessing

Each title is merged with its description to create a single text field such as for the previous models. The text and label columns are extracted from the training and test data as NumPy arrays to prepare them for model input.

In [ ]:
train_df=pd.read_csv("drive/MyDrive/train.csv")
test_df=pd.read_csv("drive/MyDrive/test.csv")

# Title and Description Concatenation + lowercasing
train_df["text"] = (train_df["Title"] + " " + train_df["Description"]).str.lower()
test_df["text"] = (test_df["Title"] + " " + test_df["Description"]).str.lower()

X_train = train_df["text"].values
y_train = train_df["Class Index"].values

X_test = test_df["text"].values
y_test = test_df["Class Index"].values

Text Cleaning by replacing newlines and tabs with spaces, collapsing multiple spaces into one, and stripping leading/trailing whitespace.

In [ ]:
def clean_text(text):
    text = re.sub(r"[\n\t]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

X_train = [clean_text(t) for t in X_train]
X_test = [clean_text(t) for t in X_test]

The training text is tokenized, each sentence is converted into a sequence of word indices, and all sequences are padded to a fixed length so they can be fed into a neural network.

In [ ]:
num_words = 200
maxlen = 250

tokenizer = Tokenizer(num_words=num_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

# Transform text into sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences
X_train_pad = pad_sequences(X_train_seq, maxlen=maxlen, padding="post")
X_test_pad = pad_sequences(X_test_seq, maxlen=maxlen, padding="post")

Class labels are shifted by 1 unit in order to ensure compatibility with neural network loss functions.

In [ ]:
y_train_enc = y_train - 1
y_test_enc = y_test - 1

### CNN architecture

The 1D CNN architecture is defined, consisting of an embedding layer, convolution with pooling for feature extraction, dense layers with dropout, and a softmax output, compiled using sparse categorical cross-entropy and accuracy metrics.

In [ ]:
model_cnn = Sequential([
    Embedding(input_dim=num_words, output_dim=128, input_length=maxlen),
    Conv1D(filters=128, kernel_size=5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(4, activation='softmax')
])

model_cnn.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)



### Training Phase & Results


The CNN is trained for up to 15 epochs using early stopping to prevent overfitting based on validation loss, and then evaluates and prints the model’s accuracy on the test set.

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history_cnn = model_cnn.fit(
    X_train_pad, y_train_enc,
    validation_split=0.1,
    epochs=5,
    batch_size=64,
    callbacks=[early_stop]
)

loss, acc = model_cnn.evaluate(X_test_pad, y_test_enc)
print("Test Accuracy CNN:", acc)

Epoch 1/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 29s 10ms/step - accuracy: 0.6543 - loss: 0.8431 - val_accuracy: 0.7361 - val_loss: 0.6712
Epoch 2/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.7611 - loss: 0.6262 - val_accuracy: 0.7441 - val_loss: 0.6614
Epoch 3/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.7769 - loss: 0.5837 - val_accuracy: 0.7497 - val_loss: 0.6511
Epoch 4/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.7884 - loss: 0.5517 - val_accuracy: 0.7488 - val_loss: 0.6586
Epoch 5/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.8005 - loss: 0.5202 - val_accuracy: 0.7464 - val_loss: 0.6542
238/238 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7633 - loss: 0.6094
Test Accuracy CNN: 0.761710524559021


## RF-Miner

### Preprocessing

Combination the Title and Description columns into a single text feature.

In [ ]:
train_df=pd.read_csv("drive/MyDrive/train.csv")
test_df=pd.read_csv("drive/MyDrive/test.csv")

train_df['text'] = train_df['Title'] + ' ' + train_df['Description']
test_df['text'] = test_df['Title'] + ' ' + test_df['Description']

We apply basic text cleaning by lowercasing, removing non-alphabetic characters, and collapsing extra whitespace for both training and test sets.

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)  # Remove non-alphabetic chars
    text = re.sub(r'\s+', ' ', text).strip()   # Collapse multiple spaces
    return text

train_df['processed_text'] = train_df['text'].apply(clean_text)
test_df['processed_text'] = test_df['text'].apply(clean_text)

The cleaned text is transformed into a TF-IDF vector space model with the top 200 features, producing numerical input arrays for training and testing the model.

In [ ]:
# TF-IDF Vectorizer with top 200 informative features
vectorizer = TfidfVectorizer(
    max_features=200,      # Keeping only 200 features as per paper
    ngram_range=(1, 2),    # unigrams + bigrams
    sublinear_tf=True
)

# Extract X (text) and y (label)
X_train = vectorizer.fit_transform(train_df['processed_text']).toarray()
X_test = vectorizer.transform(test_df['processed_text']).toarray()

y_train = train_df['Class Index'].values
y_test = test_df['Class Index'].values

### Traning Phase & Results

We set up the Random Forest parameters from the paper, then build, train, and predict with the RF-Miner classifier. Finally, the model is evaluated.

In [ ]:
# Paper Parameter Configuration
M = 200  # Total number of attributes from the VSM
nodesize = 1
mtry = 14
ntree = 35

print("Paper Parameter Configuration")
print(f"  nodesize (min_samples_leaf): {nodesize}")
print(f"  M (Total Number of Attributes): {M}")
print(f"  mtry (Subdivision Variables): {mtry}")
print(f"  ntree (Number of Trees): {ntree}")

# RF-Miner Classifier
rf_classifier = RandomForestClassifier(
    n_estimators=ntree,
    max_features=mtry,
    min_samples_leaf=nodesize,
    bootstrap=True,
    criterion='gini',
    max_depth=None,
    random_state=42,
    n_jobs=-1,
    verbose=0
)

# Training & Prediction
rf_classifier.fit(X_train, y_train)

y_pred = rf_classifier.predict(X_test)

# Evaluate the performance of the RF-Miner model
print("Accuracy RF-Miner:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Paper Parameter Configuration
  nodesize (min_samples_leaf): 1
  M (Total Number of Attributes): 200
  mtry (Subdivision Variables): 14
  ntree (Number of Trees): 35
Accuracy RF-Miner: 0.7394736842105263
              precision    recall  f1-score   support

           1       0.73      0.75      0.74      1900
           2       0.78      0.83      0.80      1900
           3       0.75      0.68      0.71      1900
           4       0.70      0.70      0.70      1900

    accuracy                           0.74      7600
   macro avg       0.74      0.74      0.74      7600
weighted avg       0.74      0.74      0.74      7600



Hyperparameter Tuning via GridSearch

In [ ]:
rf_base = RandomForestClassifier(
    bootstrap=True,
    criterion='gini',
    max_depth=None,
    random_state=42,
    n_jobs=-1
)

# grid
param_grid = {
    'n_estimators': [35, 50, 100, 200],
    'max_features': [14, 20, 30, 'sqrt'],
    'min_samples_leaf': [1, 2, 4]
}

# Grid search with 3-fold cross-validation
grid_search = GridSearchCV(
    estimator=rf_base,
    param_grid=param_grid,
    scoring='accuracy',
    cv=3,
    verbose=2,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

# Optimal parameters and performance
print("Best parameters found:", grid_search.best_params_)
best_rf = grid_search.best_estimator_
y_pred_best = best_rf.predict(X_test)

print("Optimized RF Accuracy:", accuracy_score(y_test, y_pred_best))
print(classification_report(y_test, y_pred_best))

Fitting 3 folds for each of 48 candidates, totalling 144 fits
Best parameters found: {'max_features': 14, 'min_samples_leaf': 1, 'n_estimators': 200}
Optimized RF Accuracy: 0.7482894736842105
              precision    recall  f1-score   support

           1       0.74      0.75      0.74      1900
           2       0.79      0.85      0.82      1900
           3       0.76      0.68      0.72      1900
           4       0.70      0.72      0.71      1900

    accuracy                           0.75      7600
   macro avg       0.75      0.75      0.75      7600
weighted avg       0.75      0.75      0.75      7600

